<a href="https://colab.research.google.com/github/chsuknapat/lr-crops/blob/main/AIBuilder_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install prophet
!pip install plotly==4.14.3
!pip install -U scikit-learn

     |████████████████████████████████| 71kB 3.7MB/s 
     |████████████████████████████████| 51kB 6.8MB/s 
     |████████████████████████████████| 184kB 8.9MB/s 
  Created wheel for prophet: filename=prophet-1.0.1-cp37-none-any.whl size=6640095 sha256=6b60811b8b321f01e7d5448956e6d2dda1be7e74d8af9ede7b37f433a225637f
  Stored in directory: /root/.cache/pip/wheels/92/1d/02/bc0a91bd3a84e060722075d2b0b68e875e37ec7dc49451d513
Successfully built prophet
ERROR: fbprophet 0.7.1 has requirement cmdstanpy==0.9.5, but you'll have cmdstanpy 0.9.68 which is incompatible.
  Found existing installation: cmdstanpy 0.9.5
    Uninstalling cmdstanpy-0.9.5:
      Successfully uninstalled cmdstanpy-0.9.5
     |████████████████████████████████| 13.2MB 229kB/s 
  Found existing installation: plotly 4.4.1
    Uninstalling plotly-4.4.1:
      Successfully uninstalled plotly-4.4.1
     |████████████████████████████████| 22.3MB 98kB/s 
  Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling sci

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Rain_2020_2015_clean.csv to Rain_2020_2015_clean.csv


In [2]:
import pandas as pd
import numpy as np
from prophet import Prophet
from prophet.plot import plot_plotly, plot_components_plotly
from sklearn.linear_model import LinearRegression
from sklearn.cluster import KMeans

# Rain Prediction

In [3]:
df = pd.read_csv('Rain_2020_2015_clean.csv',thousands=',')
df=df.drop(columns=['CODE', 'ZONE','DATE1'])
df['RAIN'] =pd.to_numeric(df['RAIN'])
df['DATE2'] =pd.to_datetime(df['DATE2'], format = '%Y/%m/%d')
province_out = ['OtherN','OtherNE','OtherC','OtherS','OtherNNE','OtherNC','OtherNS','OtherNEC','OtherNES','OtherCS','OtherNNEC','OtherNNES','OtherNCS','OtherNECS','WholeKingdom']
df = df[~df['PROVINCE'].isin(province_out)]
df=df.rename(columns={'DATE2' : 'ds', 'RAIN': 'y'})

df_rain = pd.DataFrame()
grouped = df.groupby('PROVINCE')

for g in grouped.groups:
     group = grouped.get_group(g)
     m = Prophet()
     m.fit(group)
     future = m.make_future_dataframe(periods=24,freq='MS')
     forecast = m.predict(future)
     #print(forecast)
     forecast_rain = forecast [['ds','yhat','yhat_lower','yhat_upper']]
     forecast_rain['PROVINCE'] = g
     df_rain = pd.concat([df_rain,forecast_rain],ignore_index=True)
     # print(df_rain)

INFO:numexpr.utils:NumExpr defaulting to 2 threads.
INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_ind

In [4]:
#len(df_rain['PROVINCE'].unique())
df_rain=df_rain[['PROVINCE','ds','yhat','yhat_lower','yhat_upper']]
df_rain.to_csv('df_rain.csv', index=False)
df_rain
# df_rainyear = df_rain.groupby(['PROVINCE' ,pd.Grouper(key='ds' ,freq='y')]).mean()
# df_rainyear = df_rainyear.reset_index()
# df_rainyear
#pd.set_option('display.max_rows' , None)#เอาหมดไม่มีย่อ


,PROVINCE,ds,yhat,yhat_lower,yhat_upper
0,AmnatChareon,2015-01-01,-8.452468,-94.580043,84.400910
1,AmnatChareon,2015-02-01,-11.143993,-94.860906,77.255765
2,AmnatChareon,2015-03-01,9.317927,-74.770494,98.692368
3,AmnatChareon,2015-04-01,40.047984,-42.871928,136.523844
4,AmnatChareon,2015-05-01,132.308874,45.625766,220.322754
...,...,...,...,...,...
7291,Yasothon,2022-08-01,354.001609,256.065038,450.730698
7292,Yasothon,2022-09-01,170.932106,79.607358,266.799485
7293,Yasothon,2022-10-01,93.949718,-9.185129,189.862983
7294,Yasothon,2022-11-01,26.105843,-72.940624,131.955839


In [5]:
plot_plotly(m, forecast)

In [6]:
plot_components_plotly(m, forecast)

# Temp Prediction

In [7]:
df = pd.read_csv('Temp_2020_2015_clean.csv',thousands=',')
df= df.drop(columns=['CODE','ZONE ', 'DATE1'])
df['TEMP'] =pd.to_numeric(df['TEMP'])
df['DATE2'] =pd.to_datetime(df['DATE2'], format = '%d/%m/%Y')
province_out = ['Bueng Kan']
df = df[~df['PROVINCE'].isin(province_out)]
df=df.rename(columns={'DATE2' : 'ds', 'TEMP': 'y'})

df_temp = pd.DataFrame()
grouped = df.groupby('PROVINCE')

for g in grouped.groups:
     group = grouped.get_group(g)
     m = Prophet()
     m.fit(group)
     future = m.make_future_dataframe(periods=24,freq='m')
     forecast = m.predict(future)
     forecast_temp = forecast [['ds','yhat','yhat_lower','yhat_upper']]
     forecast_temp['PROVINCE'] = g
     df_temp = pd.concat([df_temp,forecast_temp],ignore_index=True)
     #print(df_temp)

INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the doc

In [8]:
df_temp=df_temp[['PROVINCE','ds','yhat','yhat_lower','yhat_upper']]
#df_tempyear = df_temp.groupby(['PROVINCE' ,pd.Grouper(key='ds' ,freq='y')]).mean()
#df_tempyear = df_tempyear.reset_index()
#df_tempyear
# df_temp
#df_temp[df_temp['PROVINCE'] == 'AmnatChareon']
df_temp.to_csv('df_temp.csv', index=False)
df_temp

,PROVINCE,ds,yhat,yhat_lower,yhat_upper
0,AmnatChareon,2015-01-01,25.532592,24.205092,26.908581
1,AmnatChareon,2015-02-01,26.123221,24.760960,27.416937
2,AmnatChareon,2015-03-01,29.499186,28.219248,30.818758
3,AmnatChareon,2015-04-01,29.951251,28.560606,31.293309
4,AmnatChareon,2015-05-01,29.911131,28.634472,31.300299
...,...,...,...,...,...
7291,Yasothon,2022-07-31,27.125470,25.667870,28.549269
7292,Yasothon,2022-08-31,27.627423,26.258132,29.060847
7293,Yasothon,2022-09-30,27.847529,26.555422,29.301318
7294,Yasothon,2022-10-31,26.810958,25.487275,28.321435


In [9]:
plot_plotly(m, forecast)

In [10]:
plot_components_plotly(m, forecast)

# Linear Regression

In [11]:
df_rain = pd.read_csv('/content/df_rain.csv')
df_rain['YEAR'] = df_rain['ds'].apply(lambda x:x[:4])
df_rain['MONTH'] = df_rain['ds'].apply(lambda x:x[5:7])
df_rain = pd.DataFrame(df_rain.pivot_table(index=['PROVINCE','YEAR'], columns = 'MONTH' , values = 'yhat' )).reset_index()

df_temp = pd.read_csv('/content/df_temp.csv')
df_temp['YEAR'] = df_temp['ds'].apply(lambda x:x[:4])
df_temp['MONTH'] =df_temp['ds'].apply(lambda x:x[5:7])
df_temp = pd.DataFrame(df_temp.pivot_table(index=['PROVINCE','YEAR'], columns = 'MONTH' , values = 'yhat' )).reset_index()
# df_temp = pd.DataFrame(df_temp.pivot_table(index=['PROVINCE','YEAR'], columns = 'MONTH' )['yhat'].reset_index())

left =  df_rain.set_index(['PROVINCE' , 'YEAR' ])
right = df_temp.set_index(['PROVINCE' , 'YEAR' ])
df_rt = left.join(right, lsuffix='_rain' , rsuffix='_temp').reset_index()
df_rt['YEAR'] = pd.to_datetime(df_rt['YEAR'], format= '%Y').dt.to_period('Y')
df_rt

# left =  df_rainyear.set_index(['PROVINCE' , 'ds' ])
# right = df_tempyear.set_index(['PROVINCE' , 'ds' ])
# df_all = left.join(right, lsuffix='_rain' , rsuffix='_temp').reset_index()
# df_all['YEAR'] = df_all['ds'].dt.to_period('Y')
# df_all

MONTH,PROVINCE,YEAR,01_rain,02_rain,03_rain,04_rain,05_rain,06_rain,07_rain,08_rain,09_rain,10_rain,11_rain,12_rain,01_temp,02_temp,03_temp,04_temp,05_temp,06_temp,07_temp,08_temp,09_temp,10_temp,11_temp,12_temp
0,AmnatChareon,2015,-8.452468,-11.143993,9.317927,40.047984,132.308874,83.638318,276.665117,163.106750,248.656722,17.320519,-9.417855,-14.518534,25.532592,26.123221,29.499186,29.951251,29.911131,29.534026,27.467133,27.961936,27.458547,27.765355,26.917975,25.586040
1,AmnatChareon,2016,-0.149627,-1.197657,4.825531,64.515513,85.982813,191.816058,261.777505,162.422592,138.393380,100.429790,14.556256,-2.341514,25.128755,26.104537,29.382494,30.298239,29.608400,28.612887,28.045367,28.220324,28.126634,27.151423,26.727753,24.651856
2,AmnatChareon,2017,-5.166075,-11.602064,12.895441,62.776092,108.243195,162.263165,272.604811,168.574599,181.974547,79.779704,12.817424,-0.237369,26.295526,26.144813,29.411813,30.170891,29.695182,28.914633,27.849315,28.125565,27.889394,27.341102,26.780784,24.961059
3,AmnatChareon,2018,2.712362,-1.776458,20.813654,61.038896,130.107234,132.611768,283.986823,175.226619,225.161368,58.487822,11.239838,2.121305,25.903071,26.118474,29.442510,30.047648,29.789239,29.215653,27.651103,28.034718,27.662629,27.542469,26.841573,25.270414
4,AmnatChareon,2019,10.805284,8.113759,28.575679,59.305736,151.566626,102.896070,295.922869,182.364502,267.914474,36.578271,9.839897,4.739218,25.512470,26.103875,29.480540,29.933382,29.894012,29.517683,27.451541,27.947121,27.444508,27.752066,26.905462,25.574279
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
603,Yasothon,2018,9.230865,6.351706,20.088082,44.073648,88.028297,89.780464,202.545408,337.586070,154.516567,77.534179,9.690304,2.651754,25.332139,26.859359,29.151138,30.448031,29.717792,28.668614,28.115725,27.814359,28.112890,27.336825,26.720206,25.293709
604,Yasothon,2019,19.772295,8.839932,23.383063,59.125755,96.400649,64.007791,192.018433,392.690677,153.658319,70.151751,6.052871,7.400603,25.231656,26.561478,29.174981,30.492049,29.827532,28.834916,28.035924,27.643020,27.984359,27.458222,26.738853,25.612884
605,Yasothon,2020,30.371292,11.301082,30.267109,30.195572,85.925780,157.364834,243.256688,244.308565,170.274345,108.017229,33.818269,10.360734,25.122308,26.263445,29.119424,30.369158,29.501866,28.325078,28.257614,28.153709,28.383227,27.107375,26.691916,24.824377
606,Yasothon,2021,15.169560,20.252176,33.325512,45.369170,95.480656,131.855594,230.571933,299.051260,170.996984,101.105531,29.893825,14.582337,25.965501,29.238664,30.588743,30.049529,29.155491,27.859138,27.298303,27.741451,27.713706,26.784379,26.238878,24.876755


# Pineapple

In [12]:
df_pine = pd.read_csv('/content/Pineapple_2019_2010_clean.csv',thousands=',')
df_pine= df_pine.drop(columns=['CODE','ZONE', 'YEAR1','AREA-R','YIELD-KG/R'])
df_pine= df_pine[df_pine['PROVINCE'] != 'Bueng Kan' ]
df_pine['YIELD-T'] = pd.to_numeric(df_pine['YIELD-T'])
df_pine['YEAR'] =  pd.to_datetime(df_pine['YEAR'], format= '%Y').dt.to_period('Y')
df_pine['CROP'] = 'Pineapple'

left =  df_pine.set_index(['PROVINCE' , 'YEAR' ])
right = df_rt.set_index(['PROVINCE' , 'YEAR' ])
df_pine = left.join(right, lsuffix=' ' , rsuffix=' ').dropna().reset_index()
df_pine['PROVINCE_CROP'] = df_pine['PROVINCE']+'_pine'
df_pine.sort_values(['PROVINCE','YEAR'])

df_pineclust = df_pine.groupby(['PROVINCE_CROP']).mean().reset_index()
x = df_pineclust.drop(columns=['PROVINCE_CROP','YIELD-T']) 
seed = 42
model = KMeans(n_clusters=4,random_state = seed)
kmeans = model.fit_predict(x)

df_pineclust['CLUSTER'] = kmeans
df_pineclust['CLUSTER'] = df_pineclust['CLUSTER'].astype(str)+'_pine'
df_pineclust = df_pineclust[['PROVINCE_CROP','CLUSTER']]

left = df_pine.set_index(['PROVINCE_CROP'])
right = df_pineclust.set_index(['PROVINCE_CROP'])
df_pine = left.join(right, lsuffix=' ', rsuffix=' ').reset_index()
df_pine

# df = df[df['PROVINCE'] != 'Bueng Kan' ]
# df['YIELD-T'] = pd.to_numeric(df['YIELD-T'])   
# df['YEAR'] = pd.to_datetime(df['YEAR'], format= '%Y').dt.to_period('Y')
# #df['PROVINCE'].unique().value_count()
# left = df.set_index(['PROVINCE' , 'YEAR'])
# right = df_all.set_index(['PROVINCE' , 'YEAR'])
# df_pine = left.join(right, lsuffix='_pine' , rsuffix='_all').reset_index()
 

# grouped = df_pine.groupby('PROVINCE')
# for g in grouped.groups:
#      group = grouped.get_group(g)
#      x = group[['yhat_rain', 'yhat_temp']]
#      y = group[['YIELD-T']]
#      model = LinearRegression()
#      model.fit(x,y)
#      print(g,'R-Squared' ,model.score(x,y))
#      print(g,'Coef' , model.coef_)
#      print(g, 'intercep', model.intercept_)
#     #  model.coef_

,PROVINCE_CROP,PROVINCE,YEAR,YIELD-T,CROP,01_rain,02_rain,03_rain,04_rain,05_rain,06_rain,07_rain,08_rain,09_rain,10_rain,11_rain,12_rain,01_temp,02_temp,03_temp,04_temp,05_temp,06_temp,07_temp,08_temp,09_temp,10_temp,11_temp,12_temp,CLUSTER
0,AmnatChareon_pine,AmnatChareon,2015,196269,Pineapple,-8.452468,-11.143993,9.317927,40.047984,132.308874,83.638318,276.665117,163.106750,248.656722,17.320519,-9.417855,-14.518534,25.532592,26.123221,29.499186,29.951251,29.911131,29.534026,27.467133,27.961936,27.458547,27.765355,26.917975,25.586040,3_pine
1,AmnatChareon_pine,AmnatChareon,2016,219535,Pineapple,-0.149627,-1.197657,4.825531,64.515513,85.982813,191.816058,261.777505,162.422592,138.393380,100.429790,14.556256,-2.341514,25.128755,26.104537,29.382494,30.298239,29.608400,28.612887,28.045367,28.220324,28.126634,27.151423,26.727753,24.651856,3_pine
2,AmnatChareon_pine,AmnatChareon,2017,317478,Pineapple,-5.166075,-11.602064,12.895441,62.776092,108.243195,162.263165,272.604811,168.574599,181.974547,79.779704,12.817424,-0.237369,26.295526,26.144813,29.411813,30.170891,29.695182,28.914633,27.849315,28.125565,27.889394,27.341102,26.780784,24.961059,3_pine
3,AmnatChareon_pine,AmnatChareon,2018,343245,Pineapple,2.712362,-1.776458,20.813654,61.038896,130.107234,132.611768,283.986823,175.226619,225.161368,58.487822,11.239838,2.121305,25.903071,26.118474,29.442510,30.047648,29.789239,29.215653,27.651103,28.034718,27.662629,27.542469,26.841573,25.270414,3_pine
4,AmnatChareon_pine,AmnatChareon,2019,355759,Pineapple,10.805284,8.113759,28.575679,59.305736,151.566626,102.896070,295.922869,182.364502,267.914474,36.578271,9.839897,4.739218,25.512470,26.103875,29.480540,29.933382,29.894012,29.517683,27.451541,27.947121,27.444508,27.752066,26.905462,25.574279,3_pine
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
273,Yasothon_pine,Yasothon,2016,313660,Pineapple,13.955753,-5.114456,13.851570,13.780033,69.510242,140.949295,226.841149,227.893026,153.858806,91.601690,17.402730,-6.054805,25.132061,26.272712,29.128235,30.377482,29.509719,28.332444,28.264509,28.160116,28.389148,27.112825,26.696878,24.649283,0_pine
274,Yasothon_pine,Yasothon,2017,334562,Pineapple,-1.245979,3.836638,16.909974,28.953632,79.065118,115.440055,214.156394,282.635722,154.581445,84.689993,13.478286,-1.833202,25.426977,27.159923,29.135728,30.409663,29.611337,28.500278,28.190981,27.985633,28.246709,27.220485,26.705002,24.971253,0_pine
275,Yasothon_pine,Yasothon,2018,319787,Pineapple,9.230865,6.351706,20.088082,44.073648,88.028297,89.780464,202.545408,337.586070,154.516567,77.534179,9.690304,2.651754,25.332139,26.859359,29.151138,30.448031,29.717792,28.668614,28.115725,27.814359,28.112890,27.336825,26.720206,25.293709,0_pine
276,Yasothon_pine,Yasothon,2019,347932,Pineapple,19.772295,8.839932,23.383063,59.125755,96.400649,64.007791,192.018433,392.690677,153.658319,70.151751,6.052871,7.400603,25.231656,26.561478,29.174981,30.492049,29.827532,28.834916,28.035924,27.643020,27.984359,27.458222,26.738853,25.612884,0_pine


# Rice

In [13]:
df_rice = pd.read_csv('/content/Rice_2019_2010_clean.csv',thousands=',')
df_rice= df_rice.drop(columns=['CODE','ZONE', 'YEAR1','AREA-R','YIELD-KG/R'])
df_rice= df_rice[df_rice['PROVINCE'] != 'Bueng Kan' ]
df_rice['YIELD-T'] = pd.to_numeric(df_rice['YIELD-T'])
df_rice['YEAR'] =  pd.to_datetime(df_rice['YEAR'], format= '%Y').dt.to_period('Y')
df_rice['CROP'] = 'Rice'

left =  df_rice.set_index(['PROVINCE' , 'YEAR' ])
right = df_rt.set_index(['PROVINCE' , 'YEAR' ])
df_rice = left.join(right, lsuffix=' ' , rsuffix=' ').dropna().reset_index()
df_rice['PROVINCE_CROP'] = df_rice['PROVINCE']+'_rice'
df_rice.sort_values(['PROVINCE','YEAR'])

df_riceclust = df_rice.groupby(['PROVINCE_CROP']).mean().reset_index()
x =df_riceclust.drop(columns=['PROVINCE_CROP','YIELD-T']) 
seed = 42
model = KMeans(n_clusters=4,random_state = seed)
kmeans = model.fit_predict(x)

df_riceclust['CLUSTER'] = kmeans
df_riceclust['CLUSTER'] =df_riceclust['CLUSTER'].astype(str)+'_rice'
df_riceclust =df_riceclust[['PROVINCE_CROP','CLUSTER']]

left = df_rice.set_index(['PROVINCE_CROP'])
right = df_riceclust.set_index(['PROVINCE_CROP'])
df_rice = left.join(right, lsuffix=' ', rsuffix=' ').reset_index()
df_rice

,PROVINCE_CROP,PROVINCE,YEAR,YIELD-T,CROP,01_rain,02_rain,03_rain,04_rain,05_rain,06_rain,07_rain,08_rain,09_rain,10_rain,11_rain,12_rain,01_temp,02_temp,03_temp,04_temp,05_temp,06_temp,07_temp,08_temp,09_temp,10_temp,11_temp,12_temp,CLUSTER
0,AmnatChareon_rice,AmnatChareon,2015,316857,Rice,-8.452468,-11.143993,9.317927,40.047984,132.308874,83.638318,276.665117,163.106750,248.656722,17.320519,-9.417855,-14.518534,25.532592,26.123221,29.499186,29.951251,29.911131,29.534026,27.467133,27.961936,27.458547,27.765355,26.917975,25.586040,0_rice
1,AmnatChareon_rice,AmnatChareon,2016,313858,Rice,-0.149627,-1.197657,4.825531,64.515513,85.982813,191.816058,261.777505,162.422592,138.393380,100.429790,14.556256,-2.341514,25.128755,26.104537,29.382494,30.298239,29.608400,28.612887,28.045367,28.220324,28.126634,27.151423,26.727753,24.651856,0_rice
2,AmnatChareon_rice,AmnatChareon,2017,324963,Rice,-5.166075,-11.602064,12.895441,62.776092,108.243195,162.263165,272.604811,168.574599,181.974547,79.779704,12.817424,-0.237369,26.295526,26.144813,29.411813,30.170891,29.695182,28.914633,27.849315,28.125565,27.889394,27.341102,26.780784,24.961059,0_rice
3,AmnatChareon_rice,AmnatChareon,2018,353962,Rice,2.712362,-1.776458,20.813654,61.038896,130.107234,132.611768,283.986823,175.226619,225.161368,58.487822,11.239838,2.121305,25.903071,26.118474,29.442510,30.047648,29.789239,29.215653,27.651103,28.034718,27.662629,27.542469,26.841573,25.270414,0_rice
4,AmnatChareon_rice,AmnatChareon,2019,336807,Rice,10.805284,8.113759,28.575679,59.305736,151.566626,102.896070,295.922869,182.364502,267.914474,36.578271,9.839897,4.739218,25.512470,26.103875,29.480540,29.933382,29.894012,29.517683,27.451541,27.947121,27.444508,27.752066,26.905462,25.574279,0_rice
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
355,Yasothon_rice,Yasothon,2015,438809,Rice,3.356757,-7.575607,6.967524,42.710216,79.985110,47.592252,175.602894,376.275138,137.242780,53.736212,-10.362668,-9.014936,25.247144,26.576479,29.189541,30.506123,29.841134,28.848031,28.048568,27.655178,27.996029,27.469421,26.749565,25.623125,0_rice
356,Yasothon_rice,Yasothon,2016,442148,Rice,13.955753,-5.114456,13.851570,13.780033,69.510242,140.949295,226.841149,227.893026,153.858806,91.601690,17.402730,-6.054805,25.132061,26.272712,29.128235,30.377482,29.509719,28.332444,28.264509,28.160116,28.389148,27.112825,26.696878,24.649283,0_rice
357,Yasothon_rice,Yasothon,2017,413420,Rice,-1.245979,3.836638,16.909974,28.953632,79.065118,115.440055,214.156394,282.635722,154.581445,84.689993,13.478286,-1.833202,25.426977,27.159923,29.135728,30.409663,29.611337,28.500278,28.190981,27.985633,28.246709,27.220485,26.705002,24.971253,0_rice
358,Yasothon_rice,Yasothon,2018,459002,Rice,9.230865,6.351706,20.088082,44.073648,88.028297,89.780464,202.545408,337.586070,154.516567,77.534179,9.690304,2.651754,25.332139,26.859359,29.151138,30.448031,29.717792,28.668614,28.115725,27.814359,28.112890,27.336825,26.720206,25.293709,0_rice


# Cassava

In [14]:
df_cas = pd.read_csv('/content/Cassava_2020_2011_clean (1).csv',thousands=',')
df_cas= df_cas.drop(columns=['CODE','ZONE', 'YEAR1','AREA-R','YIELD-KG/R'])
df_cas= df_cas[df_cas['PROVINCE'] != 'Bueng Kan' ]
df_cas['YIELD-T'] = pd.to_numeric(df_cas['YIELD-T'])
df_cas['YEAR'] =  pd.to_datetime(df_cas['YEAR'], format= '%Y').dt.to_period('Y')
df_cas['CROP'] = 'Cassava'

left =  df_cas.set_index(['PROVINCE' , 'YEAR' ])
right = df_rt.set_index(['PROVINCE' , 'YEAR' ])
df_cas = left.join(right, lsuffix=' ' , rsuffix=' ').dropna().reset_index()
df_cas['PROVINCE_CROP'] = df_cas['PROVINCE']+'_cass'
df_cas.sort_values(['PROVINCE','YEAR'])

df_casclust = df_cas.groupby(['PROVINCE_CROP']).mean().reset_index()
x =df_casclust.drop(columns=['PROVINCE_CROP','YIELD-T']) 
seed = 42
model = KMeans(n_clusters=4,random_state = seed)
kmeans = model.fit_predict(x)

df_casclust['CLUSTER'] = kmeans
df_casclust['CLUSTER'] =df_casclust['CLUSTER'].astype(str)+'_cass'
df_casclust =df_casclust[['PROVINCE_CROP','CLUSTER']]

left = df_cas.set_index(['PROVINCE_CROP'])
right = df_casclust.set_index(['PROVINCE_CROP'])
df_cas = left.join(right, lsuffix=' ', rsuffix=' ').reset_index()
df_cas

,PROVINCE_CROP,PROVINCE,YEAR,YIELD-T,CROP,01_rain,02_rain,03_rain,04_rain,05_rain,06_rain,07_rain,08_rain,09_rain,10_rain,11_rain,12_rain,01_temp,02_temp,03_temp,04_temp,05_temp,06_temp,07_temp,08_temp,09_temp,10_temp,11_temp,12_temp,CLUSTER
0,AmnatChareon_cass,AmnatChareon,2015,196269,Cassava,-8.452468,-11.143993,9.317927,40.047984,132.308874,83.638318,276.665117,163.106750,248.656722,17.320519,-9.417855,-14.518534,25.532592,26.123221,29.499186,29.951251,29.911131,29.534026,27.467133,27.961936,27.458547,27.765355,26.917975,25.586040,3_cass
1,AmnatChareon_cass,AmnatChareon,2016,219535,Cassava,-0.149627,-1.197657,4.825531,64.515513,85.982813,191.816058,261.777505,162.422592,138.393380,100.429790,14.556256,-2.341514,25.128755,26.104537,29.382494,30.298239,29.608400,28.612887,28.045367,28.220324,28.126634,27.151423,26.727753,24.651856,3_cass
2,AmnatChareon_cass,AmnatChareon,2017,317478,Cassava,-5.166075,-11.602064,12.895441,62.776092,108.243195,162.263165,272.604811,168.574599,181.974547,79.779704,12.817424,-0.237369,26.295526,26.144813,29.411813,30.170891,29.695182,28.914633,27.849315,28.125565,27.889394,27.341102,26.780784,24.961059,3_cass
3,AmnatChareon_cass,AmnatChareon,2018,343245,Cassava,2.712362,-1.776458,20.813654,61.038896,130.107234,132.611768,283.986823,175.226619,225.161368,58.487822,11.239838,2.121305,25.903071,26.118474,29.442510,30.047648,29.789239,29.215653,27.651103,28.034718,27.662629,27.542469,26.841573,25.270414,3_cass
4,AmnatChareon_cass,AmnatChareon,2019,355759,Cassava,10.805284,8.113759,28.575679,59.305736,151.566626,102.896070,295.922869,182.364502,267.914474,36.578271,9.839897,4.739218,25.512470,26.103875,29.480540,29.933382,29.894012,29.517683,27.451541,27.947121,27.444508,27.752066,26.905462,25.574279,3_cass
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
273,Yasothon_cass,Yasothon,2016,313660,Cassava,13.955753,-5.114456,13.851570,13.780033,69.510242,140.949295,226.841149,227.893026,153.858806,91.601690,17.402730,-6.054805,25.132061,26.272712,29.128235,30.377482,29.509719,28.332444,28.264509,28.160116,28.389148,27.112825,26.696878,24.649283,0_cass
274,Yasothon_cass,Yasothon,2017,334562,Cassava,-1.245979,3.836638,16.909974,28.953632,79.065118,115.440055,214.156394,282.635722,154.581445,84.689993,13.478286,-1.833202,25.426977,27.159923,29.135728,30.409663,29.611337,28.500278,28.190981,27.985633,28.246709,27.220485,26.705002,24.971253,0_cass
275,Yasothon_cass,Yasothon,2018,319787,Cassava,9.230865,6.351706,20.088082,44.073648,88.028297,89.780464,202.545408,337.586070,154.516567,77.534179,9.690304,2.651754,25.332139,26.859359,29.151138,30.448031,29.717792,28.668614,28.115725,27.814359,28.112890,27.336825,26.720206,25.293709,0_cass
276,Yasothon_cass,Yasothon,2019,347932,Cassava,19.772295,8.839932,23.383063,59.125755,96.400649,64.007791,192.018433,392.690677,153.658319,70.151751,6.052871,7.400603,25.231656,26.561478,29.174981,30.492049,29.827532,28.834916,28.035924,27.643020,27.984359,27.458222,26.738853,25.612884,0_cass


# Maize

In [15]:
df_maize = pd.read_csv('/content/Maize_2019_2010.csv',thousands=',')
df_maize= df_maize.drop(columns=['CODE','ZONE', 'YEAR1','AREA-R','YIELD-KG/R'])
df_maize= df_maize[df_maize['PROVINCE'] != 'Bueng Kan' ]
df_maize['YIELD-T'] = pd.to_numeric(df_maize['YIELD-T'])
df_maize['YEAR'] =  pd.to_datetime(df_maize['YEAR'], format= '%Y').dt.to_period('Y')
df_maize['CROP'] = 'Maize'

left =  df_maize.set_index(['PROVINCE' , 'YEAR' ])
right = df_rt.set_index(['PROVINCE' , 'YEAR' ])
df_maize = left.join(right, lsuffix=' ' , rsuffix=' ').dropna().reset_index()
df_maize['PROVINCE_CROP'] = df_maize['PROVINCE']+'_maze'
df_maize.sort_values(['PROVINCE','YEAR'])

df_maizeclust = df_maize.groupby(['PROVINCE_CROP']).mean().reset_index()
x =df_maizeclust.drop(columns=['PROVINCE_CROP','YIELD-T']) 
seed = 42
model = KMeans(n_clusters=4,random_state = seed)
kmeans = model.fit_predict(x)

df_maizeclust['CLUSTER'] = kmeans
df_maizeclust['CLUSTER'] =df_maizeclust['CLUSTER'].astype(str)+'_maze'
df_maizeclust =df_maizeclust[['PROVINCE_CROP','CLUSTER']]

left = df_maize.set_index(['PROVINCE_CROP'])
right =df_maizeclust.set_index(['PROVINCE_CROP'])
df_maize = left.join(right, lsuffix=' ', rsuffix=' ').reset_index()
df_maize

,PROVINCE_CROP,PROVINCE,YEAR,YIELD-T,CROP,01_rain,02_rain,03_rain,04_rain,05_rain,06_rain,07_rain,08_rain,09_rain,10_rain,11_rain,12_rain,01_temp,02_temp,03_temp,04_temp,05_temp,06_temp,07_temp,08_temp,09_temp,10_temp,11_temp,12_temp,CLUSTER
0,AmnatChareon_maze,AmnatChareon,2018,523.0,Maize,2.712362,-1.776458,20.813654,61.038896,130.107234,132.611768,283.986823,175.226619,225.161368,58.487822,11.239838,2.121305,25.903071,26.118474,29.442510,30.047648,29.789239,29.215653,27.651103,28.034718,27.662629,27.542469,26.841573,25.270414,1_maze
1,AmnatChareon_maze,AmnatChareon,2019,485.0,Maize,10.805284,8.113759,28.575679,59.305736,151.566626,102.896070,295.922869,182.364502,267.914474,36.578271,9.839897,4.739218,25.512470,26.103875,29.480540,29.933382,29.894012,29.517683,27.451541,27.947121,27.444508,27.752066,26.905462,25.574279,1_maze
2,BuriRum_maze,BuriRum,2018,656.0,Maize,4.703562,9.791590,47.871254,80.897689,115.333175,99.766315,136.968665,180.627502,233.429558,110.700715,25.522758,4.306154,25.172023,26.545801,29.260178,29.704129,29.330444,28.290987,28.478206,27.515937,27.448703,26.659494,25.599577,24.837195,3_maze
3,BuriRum_maze,BuriRum,2019,350.0,Maize,10.447524,10.623976,49.270896,95.876699,116.117177,99.148147,123.108027,197.834537,253.544337,103.645828,23.693576,9.932726,25.118441,26.440650,29.401259,29.532863,29.323877,28.436817,28.805367,27.378310,27.262694,26.767590,25.492421,25.228769,3_maze
4,Chachoengsao_maze,Chachoengsao,2015,2057.0,Maize,36.554880,25.192649,64.511225,108.184841,217.013576,144.447315,170.811859,111.659414,280.816776,168.520200,75.966135,13.977254,25.811819,26.905821,27.702191,28.699026,28.812390,28.245228,28.656463,27.960630,27.616813,27.632967,27.131294,26.433513,3_maze
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190,Uttaradit_maze,Uttaradit,2017,130607.0,Maize,34.156561,9.860035,3.843067,41.483869,120.549489,75.439552,119.157112,215.475243,152.356186,82.748965,13.517028,11.327165,26.433712,25.989598,29.681427,31.183104,30.700918,29.476218,28.443788,28.476630,28.682419,28.130700,27.465025,24.928227,3_maze
191,Uttaradit_maze,Uttaradit,2018,144186.0,Maize,25.629393,7.365476,12.278078,48.333655,106.943842,88.820259,125.591489,216.622750,135.879096,92.946043,25.459075,17.575052,25.886503,26.203491,29.723973,31.038593,30.600439,29.663452,28.330572,28.493569,28.650882,28.334136,27.706440,25.115940,3_maze
192,Uttaradit_maze,Uttaradit,2019,132277.0,Maize,16.934868,5.055531,20.811014,54.970102,93.426660,102.442478,131.826287,217.414960,119.529450,103.543451,37.406364,23.503664,25.518724,26.624837,29.980553,31.130842,30.762997,30.088088,28.431737,28.705706,28.794413,28.691074,28.073876,25.403404,3_maze
193,Yasothon_maze,Yasothon,2018,719.0,Maize,9.230865,6.351706,20.088082,44.073648,88.028297,89.780464,202.545408,337.586070,154.516567,77.534179,9.690304,2.651754,25.332139,26.859359,29.151138,30.448031,29.717792,28.668614,28.115725,27.814359,28.112890,27.336825,26.720206,25.293709,1_maze


# All Crop

In [16]:
df_all = pd.concat([df_pine,df_rice,df_cas,df_maize], axis=0).reset_index(drop=True)
df_all.to_csv('df_all.csv' , index=False)
df_all#.head()
#df_all.isnull().sum()

,PROVINCE_CROP,PROVINCE,YEAR,YIELD-T,CROP,01_rain,02_rain,03_rain,04_rain,05_rain,06_rain,07_rain,08_rain,09_rain,10_rain,11_rain,12_rain,01_temp,02_temp,03_temp,04_temp,05_temp,06_temp,07_temp,08_temp,09_temp,10_temp,11_temp,12_temp,CLUSTER
0,AmnatChareon_pine,AmnatChareon,2015,196269.0,Pineapple,-8.452468,-11.143993,9.317927,40.047984,132.308874,83.638318,276.665117,163.106750,248.656722,17.320519,-9.417855,-14.518534,25.532592,26.123221,29.499186,29.951251,29.911131,29.534026,27.467133,27.961936,27.458547,27.765355,26.917975,25.586040,3_pine
1,AmnatChareon_pine,AmnatChareon,2016,219535.0,Pineapple,-0.149627,-1.197657,4.825531,64.515513,85.982813,191.816058,261.777505,162.422592,138.393380,100.429790,14.556256,-2.341514,25.128755,26.104537,29.382494,30.298239,29.608400,28.612887,28.045367,28.220324,28.126634,27.151423,26.727753,24.651856,3_pine
2,AmnatChareon_pine,AmnatChareon,2017,317478.0,Pineapple,-5.166075,-11.602064,12.895441,62.776092,108.243195,162.263165,272.604811,168.574599,181.974547,79.779704,12.817424,-0.237369,26.295526,26.144813,29.411813,30.170891,29.695182,28.914633,27.849315,28.125565,27.889394,27.341102,26.780784,24.961059,3_pine
3,AmnatChareon_pine,AmnatChareon,2018,343245.0,Pineapple,2.712362,-1.776458,20.813654,61.038896,130.107234,132.611768,283.986823,175.226619,225.161368,58.487822,11.239838,2.121305,25.903071,26.118474,29.442510,30.047648,29.789239,29.215653,27.651103,28.034718,27.662629,27.542469,26.841573,25.270414,3_pine
4,AmnatChareon_pine,AmnatChareon,2019,355759.0,Pineapple,10.805284,8.113759,28.575679,59.305736,151.566626,102.896070,295.922869,182.364502,267.914474,36.578271,9.839897,4.739218,25.512470,26.103875,29.480540,29.933382,29.894012,29.517683,27.451541,27.947121,27.444508,27.752066,26.905462,25.574279,3_pine
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1106,Uttaradit_maze,Uttaradit,2017,130607.0,Maize,34.156561,9.860035,3.843067,41.483869,120.549489,75.439552,119.157112,215.475243,152.356186,82.748965,13.517028,11.327165,26.433712,25.989598,29.681427,31.183104,30.700918,29.476218,28.443788,28.476630,28.682419,28.130700,27.465025,24.928227,3_maze
1107,Uttaradit_maze,Uttaradit,2018,144186.0,Maize,25.629393,7.365476,12.278078,48.333655,106.943842,88.820259,125.591489,216.622750,135.879096,92.946043,25.459075,17.575052,25.886503,26.203491,29.723973,31.038593,30.600439,29.663452,28.330572,28.493569,28.650882,28.334136,27.706440,25.115940,3_maze
1108,Uttaradit_maze,Uttaradit,2019,132277.0,Maize,16.934868,5.055531,20.811014,54.970102,93.426660,102.442478,131.826287,217.414960,119.529450,103.543451,37.406364,23.503664,25.518724,26.624837,29.980553,31.130842,30.762997,30.088088,28.431737,28.705706,28.794413,28.691074,28.073876,25.403404,3_maze
1109,Yasothon_maze,Yasothon,2018,719.0,Maize,9.230865,6.351706,20.088082,44.073648,88.028297,89.780464,202.545408,337.586070,154.516567,77.534179,9.690304,2.651754,25.332139,26.859359,29.151138,30.448031,29.717792,28.668614,28.115725,27.814359,28.112890,27.336825,26.720206,25.293709,1_maze


# Regression

In [17]:
final_data =  pd.read_csv('/content/df_all.csv',thousands=',')
final_dic = {}
for i in final_data['CLUSTER'].unique():
    group = final_data[(final_data['CLUSTER'] == i )]
    x = group.drop(['PROVINCE','YEAR','YIELD-T','CROP', 'PROVINCE_CROP','CLUSTER'], axis=1)
    y = group[['YIELD-T']]
    coef_dic = {}
    model = LinearRegression()
    model.fit(x,y)
    coef = model.coef_
    intercept = model.intercept_
    coef_dic['01_rain'] = coef[0,0]
    coef_dic['02_rain'] = coef[0,1]
    coef_dic['03_rain'] = coef[0,2]
    coef_dic['04_rain'] = coef[0,3]
    coef_dic['05_rain'] = coef[0,4]
    coef_dic['06_rain'] = coef[0,5]
    coef_dic['07_rain'] = coef[0,6]
    coef_dic['08_rain'] = coef[0,7]
    coef_dic['09_rain'] = coef[0,8]
    coef_dic['10_rain'] = coef[0,9]
    coef_dic['11_rain'] = coef[0,10]
    coef_dic['12_rain'] = coef[0,11]
    coef_dic['01_temp'] = coef[0,12]
    coef_dic['02_temp'] = coef[0,13]
    coef_dic['03_temp'] = coef[0,14]
    coef_dic['04_temp'] = coef[0,15]
    coef_dic['05_temp'] = coef[0,16]
    coef_dic['06_temp'] = coef[0,17]
    coef_dic['07_temp'] = coef[0,18]
    coef_dic['08_temp'] = coef[0,19]
    coef_dic['09_temp'] = coef[0,20]
    coef_dic['10_temp'] = coef[0,21]
    coef_dic['11_temp'] = coef[0,22]
    coef_dic['12_temp'] = coef[0,23]
    coef_dic['intercept'] = intercept[0]
    final_dic[i] = coef_dic
print(final_dic)   

{'3_pine': {'01_rain': -13048.025309480721, '02_rain': 16984.65699092139, '03_rain': 205.10467235057592, '04_rain': 764.0254948707728, '05_rain': 10981.788282601696, '06_rain': -6054.4090856737275, '07_rain': -8549.70453630194, '08_rain': -10771.540717376629, '09_rain': -414.4013542420487, '10_rain': 9660.787626238938, '11_rain': -9889.993195341849, '12_rain': 8890.011882848657, '01_temp': 48648.76435121138, '02_temp': 47532.23028572925, '03_temp': -197100.8551095023, '04_temp': 652421.1083563407, '05_temp': -2120051.900798221, '06_temp': 268349.3447608033, '07_temp': -162161.67167646115, '08_temp': 32254.76776796904, '09_temp': -362556.7260527656, '10_temp': 1526428.9606202368, '11_temp': -219081.12003785558, '12_temp': 372840.8687057313, 'intercept': 10479589.0255736}, '1_pine': {'01_rain': 1237.6111360389211, '02_rain': 1864.7630047042162, '03_rain': 97548.29738016748, '04_rain': -67457.62027636755, '05_rain': -30381.126899200146, '06_rain': -92097.075421872, '07_rain': 46406.831213